### Import Required Libraries and Set Up Environment Variables

In [80]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [81]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")


### Access the New York Times API

In [82]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
url = f"{url}api-key={nyt_api_key}&fq={filter_query}&sort={sort}&begin_date={begin_date}&end_date={end_date}&fl={field_list}"


In [83]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0, 20):
    # create query with a page number
    query_url = f"{url}&page={page}"
    # API results show 10 articles at a time
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:        
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
            
        # Print the page that was just retrieved
        print(f"Retrieving page {page} of reviews...")    
    except IndexError:
        # Print the page number that had no results then break from the loop
        print(f"No results on page {page}")
        break


Retrieving page 0 of reviews...
Retrieving page 1 of reviews...
Retrieving page 2 of reviews...
Retrieving page 3 of reviews...
Retrieving page 4 of reviews...
Retrieving page 5 of reviews...
Retrieving page 6 of reviews...
Retrieving page 7 of reviews...
Retrieving page 8 of reviews...
Retrieving page 9 of reviews...
Retrieving page 10 of reviews...
Retrieving page 11 of reviews...
Retrieving page 12 of reviews...
Retrieving page 13 of reviews...
Retrieving page 14 of reviews...
Retrieving page 15 of reviews...
Retrieving page 16 of reviews...
Retrieving page 17 of reviews...
Retrieving page 18 of reviews...
Retrieving page 19 of reviews...


In [84]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [85]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [86]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

reviews_df["Title"] = reviews_df["headline.main"].apply(lambda x: x[x.find("\u2018")+1:x.find("\u2019 Review")])
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,Title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [87]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df["keywords"].apply(extract_keywords)
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,Title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [88]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = []
titles = reviews_df["Title"].to_list()
titles[:5]

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children']

### Access The Movie Database API

In [89]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [90]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        time.sleep(1)
        print(f"Sleeping for 2 seconds... after {request_counter} requests")

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    movie_response = requests.get(url + title + tmdb_key_string).json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:        
        # Get movie id
        movie_id = movie_response["results"][0]["id"]

        # Make a request for a the full movie details
        
        # Execute "GET" request with url
        movie_details = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}").json()

        # Extract the genre names into a list
        genre_names = []
        for genre in movie_details["genres"]:
            genre_names.append(genre["name"])

        # Extract the spoken_languages' English name into a list
        spoken_languages = []
        for language in movie_details["spoken_languages"]:
            spoken_languages.append(language["english_name"])

        # Extract the production_countries' name into a list
        production_countries = []
        for country in movie_details["production_countries"]:
            production_countries.append(country["name"])

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list             
        movie_details_dict = {"Title": movie_details["title"],
                              "Original Title": movie_details["title"], 
                              "Budget": movie_details["title"], 
                              "Original Language": movie_details["title"], 
                              "Homepage": movie_details["title"], 
                              "Overview": movie_details["title"], 
                              "Popularity": movie_details["title"], 
                              "Runtime": movie_details["title"], 
                              "Revenue": movie_details["title"], 
                              "Release Date": movie_details["title"], 
                              "Vote Average": movie_details["title"], 
                              "Vote Count": movie_details["title"],
                              "Genres": genre_names,
                              "Spoken Languages": spoken_languages,
                              "Production Countries": production_countries}  
        
        tmdb_movies_list.append(movie_details_dict)
        
        print(f"Adding {title} to the list...")
                
    except IndexError:
        # Print out the title that was found
        print(f"{title} not found")
        continue


Adding The Attachment Diaries to the list...
What’s Love Got to Do With It?’ Probably a Lo not found
Adding You Can Live Forever to the list...
Adding A Tourist’s Guide to Love to the list...
Adding Other People’s Children to the list...
Adding One True Loves to the list...
Adding The Lost Weekend: A Love Story to the list...
Adding A Thousand and One to the list...
Adding Your Place or Mine to the list...
Adding Love in the Time of Fentanyl to the list...
Adding Pamela, a Love Story to the list...
Adding In From the Side to the list...
Adding After Love to the list...
Adding Alcarràs to the list...
Adding Nelly & Nadine to the list...
Adding Lady Chatterley’s Lover to the list...
Adding The Sound of Christmas to the list...
Adding The Inspection to the list...
Adding Bones and All to the list...
Adding My Policeman to the list...
Adding About Fate to the list...
Adding Waiting for Bojangles to the list...
Adding I Love My Dad to the list...
Adding A Love Song to the list...
Adding Alo

In [91]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
#print(json.dumps(tmdb_movies_list[:5], indent=4))
#tmdb_movies_list[:5]
print(json.dumps(tmdb_movies_list[:5], indent=4))


[
    {
        "Title": "The Attachment Diaries",
        "Original Title": "The Attachment Diaries",
        "Budget": "The Attachment Diaries",
        "Original Language": "The Attachment Diaries",
        "Homepage": "The Attachment Diaries",
        "Overview": "The Attachment Diaries",
        "Popularity": "The Attachment Diaries",
        "Runtime": "The Attachment Diaries",
        "Revenue": "The Attachment Diaries",
        "Release Date": "The Attachment Diaries",
        "Vote Average": "The Attachment Diaries",
        "Vote Count": "The Attachment Diaries",
        "Genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "Spoken Languages": [
            "Spanish"
        ],
        "Production Countries": [
            "Argentina"
        ]
    },
    {
        "Title": "You Can Live Forever",
        "Original Title": "You Can Live Forever",
        "Budget": "You Can Live Forever",
        "Original Langu

In [92]:
# Convert the results to a DataFrame
tmdb_movies_list_df = pd.DataFrame(tmdb_movies_list)
tmdb_movies_list_df.head()

,Title,Original Title,Budget,Original Language,Homepage,Overview,Popularity,Runtime,Revenue,Release Date,Vote Average,Vote Count,Genres,Spoken Languages,Production Countries
0,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,A Tourist's Guide to Love,A Tourist's Guide to Love,A Tourist's Guide to Love,A Tourist's Guide to Love,A Tourist's Guide to Love,A Tourist's Guide to Love,A Tourist's Guide to Love,A Tourist's Guide to Love,A Tourist's Guide to Love,A Tourist's Guide to Love,A Tourist's Guide to Love,A Tourist's Guide to Love,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,Other People's Children,Other People's Children,Other People's Children,Other People's Children,Other People's Children,Other People's Children,Other People's Children,Other People's Children,Other People's Children,Other People's Children,Other People's Children,Other People's Children,"[Drama, Comedy]","[French, English]",[France]
4,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]"


### Merge and Clean the Data for Export

In [93]:
# Merge the New York Times reviews and TMDB DataFrames on title
movies_list_df = pd.merge(reviews_df, tmdb_movies_list_df, on="Title", how="inner")
movies_list_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,Overview,Popularity,Runtime,Revenue,Release Date,Vote Average,Vote Count,Genres,Spoken Languages,Production Countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,[Documentary],[English],[United States of America]
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,A Thousand and One,A Thousand and One,A Thousand and One,A Thousand and One,A Thousand and One,A Thousand and One,A Thousand and One,"[Drama, Crime]","[English, Spanish, Portuguese]",[United States of America]


In [94]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["Genres", "Spoken Languages", "Production Countries"]

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    movies_list_df[column] = movies_list_df[column].astype(str)

    # Loop through characters to remove
    for character in characters_to_remove:
        # Replace the character with an empty string
        movies_list_df[column] = movies_list_df[column].str.replace(character, "")

# Display the fixed DataFrame
movies_list_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,Overview,Popularity,Runtime,Revenue,Release Date,Vote Average,Vote Count,Genres,Spoken Languages,Production Countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,"Romance, Comedy, Drama","English, Spanish","Czech Republic, United States of America"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,Documentary,English,United States of America
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,A Thousand and One,A Thousand and One,A Thousand and One,A Thousand and One,A Thousand and One,A Thousand and One,A Thousand and One,"Drama, Crime","English, Spanish, Portuguese",United States of America


In [95]:
# Drop "byline.person" column
movies_list_df.drop(columns=["byline.person"], inplace=True)
movies_list_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,Overview,Popularity,Runtime,Revenue,Release Date,Vote Average,Vote Count,Genres,Spoken Languages,Production Countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,"Romance, Comedy, Drama","English, Spanish","Czech Republic, United States of America"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,Documentary,English,United States of America
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,A Thousand and One,A Thousand and One,A Thousand and One,A Thousand and One,A Thousand and One,A Thousand and One,A Thousand and One,"Drama, Crime","English, Spanish, Portuguese",United States of America


In [96]:
# Delete duplicate rows and reset index
movies_list_df.drop_duplicates(inplace=True)
movies_list_df.reset_index(drop=True, inplace=True)
movies_list_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,Overview,Popularity,Runtime,Revenue,Release Date,Vote Average,Vote Count,Genres,Spoken Languages,Production Countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,You Can Live Forever,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,One True Loves,"Romance, Comedy, Drama","English, Spanish","Czech Republic, United States of America"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,Documentary,English,United States of America
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,A Thousand and One,A Thousand and One,A Thousand and One,A Thousand and One,A Thousand and One,A Thousand and One,A Thousand and One,"Drama, Crime","English, Spanish, Portuguese",United States of America


In [97]:
# Export data to CSV without the index
movies_list_df.to_csv("output/movies_list.csv", index=False, mode="w")